In [256]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [312]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb', 'jan', 'mar']
sheet = 'Sheet'

In [4]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [282]:
df = pd.read_excel(path_to_data + 'jan.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_jan', 'job':'job_jan'})
sums


,inn,snils,sum_jan,job_jan
0,5003021368,001-191-281 85,106626.88,Учитель
1,5003021368,001-244-107 65,109394.87,Учитель
2,5003021368,001-263-234 77,74520.00,Воспитатель
3,5003021368,001-322-594 80,83013.52,Специалист
4,5003021368,001-519-663 10,142589.38,Учитель
...,...,...,...,...
113367,9718071371,196-882-391 45,77596.00,Бухгалтер
113368,9718071371,198-036-430 93,110851.00,Бухгалтер
113369,9718071371,199-418-719 35,131067.00,Бухгалтер
113370,9718071371,205-412-630 13,189077.50,Начальник группы


In [284]:
df = pd.read_excel(path_to_data + 'feb.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_feb', 'job':'job_feb'})

In [283]:
res = sums

In [286]:
res

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
0,5003021368,001-191-281 85,106626.88,Учитель,139753.92,Учитель
1,5003021368,001-244-107 65,109394.87,Учитель,109541.13,Учитель
2,5003021368,001-263-234 77,74520.00,Воспитатель,81178.00,Воспитатель
3,5003021368,001-322-594 80,83013.52,Специалист,75815.00,Специалист
4,5003021368,001-519-663 10,142589.38,Учитель,163810.32,Учитель
...,...,...,...,...,...,...
131871,9718071371,166-448-762 08,NaN,NaN,110851.00,Бухгалтер
131872,9718071371,173-386-417 88,NaN,NaN,101079.63,Бухгалтер
131873,9718071371,175-876-580 36,NaN,NaN,110851.00,Бухгалтер
131874,9718071371,176-772-921 22,NaN,NaN,110851.00,Инженер-сметчик


In [285]:
res = res.merge(sums, how= 'outer', on=['inn', 'snils'])

In [309]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [310]:
def create_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data(months[i])
        else:
            res = pd.merge(res, load_data(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [313]:
res = create_res(months)

In [141]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [142]:
print_df(res[(res.inn == 5003021368) & (res.job_feb == 'Учитель')], 'teach_check')

In [287]:
res[(res.inn == 9718071371)]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
112515,9718071371,001-102-623 46,147930.0,ведущий Бухгалтер,157930.00,ведущий Бухгалтер
112516,9718071371,001-184-402 83,177930.0,ведущий Бухгалтер,177930.00,ведущий Бухгалтер
112517,9718071371,001-200-216 33,115851.0,Бухгалтер,142896.32,Бухгалтер
112518,9718071371,001-208-598 92,96078.0,Бухгалтер,117705.00,Бухгалтер
112519,9718071371,001-257-058 90,114851.0,Бухгалтер,110851.00,Бухгалтер
...,...,...,...,...,...,...
131871,9718071371,166-448-762 08,NaN,NaN,110851.00,Бухгалтер
131872,9718071371,173-386-417 88,NaN,NaN,101079.63,Бухгалтер
131873,9718071371,175-876-580 36,NaN,NaN,110851.00,Бухгалтер
131874,9718071371,176-772-921 22,NaN,NaN,110851.00,Инженер-сметчик


In [48]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [315]:
res['szp'] = res.apply(lambda row: szp(row, months), axis= 1)

In [55]:
depr_month_inc = 84100.0

In [20]:
df = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД')

In [24]:
df

,Должность,УГД,Педагогический,Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми
0,Главный бухгалтер,Административно-управленческий персонал,0,0
1,Директор,Административно-управленческий персонал,0,0
2,Директор филиала,Административно-управленческий персонал,0,0
3,Заведующий,Административно-управленческий персонал,0,0
4,Заместитель главного бухгалтера,Административно-управленческий персонал,0,0
...,...,...,...,...
291,Эксперт,Прочий персонал (включая иной административный...,0,0
292,Электромеханик,Прочий персонал (включая иной административный...,0,0
293,Старший юрисконсульт,Прочий персонал (включая иной административный...,0,0
294,Ведущий юрисконсульт,Прочий персонал (включая иной административный...,0,0


In [23]:
ped = df[df['Педагогический'] == 1]['Должность'].tolist()
ped

['Ассистент',
 'Декан факультета',
 'Директор института',
 'Доцент',
 'Заведующий кафедрой',
 'Профессор',
 'Преподаватель',
 'Старший преподаватель',
 'Воспитатель',
 'Инструктор-методист',
 'Инструктор по труду',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Методист',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Преподаватель-организатор основ безопасности жизнедеятельности',
 'Руководитель физического воспитания',
 'Социальный педагог',
 'Старший вожатый',
 'Старший воспитатель',
 'Старший инструктор-методист',
 'Старший методист',
 'Старший педагог дополнительного образования',
 'Старший педагог-психолог',
 'Старший тренер-преподаватель',
 'Тренер-преподаватель',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [25]:
isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
isp

['Преподаватель',
 'Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед',
 'Помощник воспитателя',
 'Младший воспитатель']

In [51]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [224]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [319]:
res['szp_ped'] = res.apply(lambda row: szp_ped(row, months), axis=1)

In [320]:
res['szp_teach'] = res.apply(lambda row: szp_teach(row, months), axis=1)

In [321]:
res

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach
0,5003021368,001-191-281 85,139753.92,1.0,Учитель,106626.88,1.0,Учитель,NaN,NaN,NaN,NaN,NaN,NaN
1,5003021368,001-244-107 65,109541.13,1.0,Учитель,109394.87,1.0,Учитель,109077.82,1.0,Учитель,109337.940000,109337.940000,109337.940000
2,5003021368,001-263-234 77,81178.00,1.0,Воспитатель,74520.00,1.0,Воспитатель,87744.00,1.0,Воспитатель,81147.333333,81147.333333,NaN
3,5003021368,001-322-594 80,75815.00,1.0,Специалист,83013.52,1.0,Специалист,81798.18,1.0,Специалист,80208.900000,NaN,NaN
4,5003021368,001-519-663 10,163810.32,1.0,Учитель,142589.38,1.0,Учитель,151892.97,1.0,Учитель,152764.223333,152764.223333,152764.223333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152657,9718071371,171-577-056 80,NaN,NaN,NaN,NaN,NaN,NaN,94223.00,1.0,Бухгалтер,94223.000000,NaN,NaN
152658,9718071371,174-487-457 12,NaN,NaN,NaN,NaN,NaN,NaN,527029.00,1.0,Начальник управления,527029.000000,NaN,NaN
152659,9718071371,181-370-565 64,NaN,NaN,NaN,NaN,NaN,NaN,197930.00,1.0,Ведущий специалист,197930.000000,NaN,NaN
152660,9718071371,183-851-618 98,NaN,NaN,NaN,NaN,NaN,NaN,120851.00,1.0,Экономист,120851.000000,NaN,NaN


In [322]:
val_5 = res[['inn', 'szp_ped']].groupby('inn').mean()
val_5['value'] = val_5['szp_ped'] / depr_month_inc
val_5

,szp_ped,value
inn,,
5003021368,102889.981904,1.223424
5003021495,111041.224471,1.320347
5003021640,111262.970998,1.322984
5003096290,100199.192770,1.191429
5030032182,108341.522539,1.288246
...,...,...
7743940590,129138.584406,1.535536
9705101759,NaN,NaN
9715206976,106040.984879,1.260892


In [323]:
print_df(val_5, 'val_5')
print_df(res[res.szp_ped.isna() == False][['inn', 'snils', 'szp_ped']], 'val_5_ppl')

In [324]:
val_6 = res[['inn', 'szp_ped']].groupby('inn').quantile(0.5).rename(columns={'szp_ped': 'q'})
val_6 = pd.merge(val_6, res[['inn', 'snils', 'szp_ped']], how='right', on='inn')
val_6_out = val_6
val_6 = val_6[val_6.szp_ped <= val_6.q][['inn', 'szp_ped']].groupby('inn').mean()
val_6

,szp_ped
inn,
5003021368,76511.005960
5003021495,84280.916283
5003021640,80325.153030
5003096290,72094.784913
5030032182,75085.680529
...,...
7743937357,97618.398838
7743939315,75548.254524
7743940590,102726.402673


In [325]:
print_df(val_6, 'val_6')
print_df(val_6_out, 'val_6_ppl')

In [227]:
res[(res.job_jan == 'Учитель') & (res.szp_teach == np.nan)]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped,szp_teach


In [326]:
val_7 = pd.merge(res[['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt1'}), 
res[res.szp_teach >= 84100][['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt2'}), how='outer', on='inn')
val_7['val'] = val_7.cnt2 / val_7.cnt1
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5003021368,301,298.0,0.990033
5003096290,159,158.0,0.993711
5030032182,66,65.0,0.984848
5074019220,103,102.0,0.990291
5074045703,180,174.0,0.966667
...,...,...,...
7743026202,81,79.0,0.975309
7743868110,59,52.0,0.881356
7743928539,17,14.0,0.823529


In [327]:
print_df(val_7, 'val_7')
print_df(res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], 'val_7_ppl')

In [206]:
def q_cnt1(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] <= row['0.1'] else np.nan

In [207]:
def q_cnt2(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] >= row['0.9'] else np.nan

In [328]:
q01 = res[['inn','szp']].groupby('inn').quantile(0.1).rename(columns={'szp':'0.1'})
q09 = res[['inn','szp']].groupby('inn').quantile(0.9).rename(columns={'szp':'0.9'})
q = pd.merge(q01, q09, how='right', on='inn')
qm = pd.merge(res[['inn', 'szp']], q, how='left', on='inn')
qm['cnt1'] = qm.apply(lambda row: q_cnt1(row), axis=1)
qm['cnt2'] = qm.apply(lambda row: q_cnt2(row), axis=1)
qm_pers = pd.merge(qm[qm['szp'].isna() == False], res[['inn','szp', 'snils']], how='left', on=['inn', 'szp']).drop_duplicates(['inn', 'snils'])
qm = qm[['inn', 'cnt1', 'cnt2']].groupby('inn').mean()
qm['ans'] = qm.cnt2 / qm.cnt1
qm[qm.ans > 3.7]

,cnt1,cnt2,ans
inn,,,
7701293580,71844.047619,294970.708095,4.105708
7702061938,80552.333333,339784.222222,4.218180
7702833359,56040.328200,209469.727533,3.737839
7704040242,83877.770476,364994.284762,4.351502
7704118139,54532.141667,232165.081000,4.257399
...,...,...,...
7737066359,67278.787879,262367.497273,3.899706
7743018900,52571.226842,201066.760702,3.824654
7743021236,61614.641282,238306.869231,3.867699


In [197]:
q01[q01['0.1'].isna()]

,0.1
inn,


In [329]:
print_df(qm, 'dezil')
print_df(qm_pers, 'dezil_pers')

In [330]:
cut_off = res[['inn', 'szp']].groupby('inn').count().rename(columns={'szp': 'cnt'})
cut_off.cnt = np.ceil(cut_off.cnt / 10)
cut_off = cut_off.to_dict()['cnt']
cut_off

{5003021368: 78.0,
 5003021495: 84.0,
 5003021640: 47.0,
 5003096290: 44.0,
 5030032182: 22.0,
 5030032376: 44.0,
 5051005503: 28.0,
 5051005623: 13.0,
 5051005670: 79.0,
 5074019220: 25.0,
 5074019252: 33.0,
 5074045703: 47.0,
 7701032028: 3.0,
 7701050524: 14.0,
 7701113894: 5.0,
 7701293580: 7.0,
 7701375995: 6.0,
 7701377657: 18.0,
 7701384943: 3.0,
 7701406844: 6.0,
 7701905508: 24.0,
 7702058364: 14.0,
 7702061938: 3.0,
 7702764296: 15.0,
 7702797527: 25.0,
 7702833359: 25.0,
 7703021039: 6.0,
 7703112310: 8.0,
 7703261626: 12.0,
 7703363890: 22.0,
 7703379868: 3.0,
 7703611221: 16.0,
 7703738796: 18.0,
 7703742922: 33.0,
 7703776689: 9.0,
 7703808740: 11.0,
 7704040242: 7.0,
 7704118121: 5.0,
 7704118139: 10.0,
 7704178709: 14.0,
 7704191153: 7.0,
 7704222919: 15.0,
 7704222926: 35.0,
 7704231631: 28.0,
 7704234760: 22.0,
 7704271031: 1.0,
 7704853583: 17.0,
 7705020295: 12.0,
 7705041520: 3.0,
 7705052410: 6.0,
 7705399348: 25.0,
 7705480398: 25.0,
 7705513678: 23.0,
 770551373

In [333]:
ppl = res[['inn', 'szp']].sort_values('szp').dropna().groupby('inn')
keys = ppl.groups.keys()
val_1_7 = pd.DataFrame({'inn': [], 'val': []})
val_1_7

,inn,val


In [334]:
for key in keys:
    top = ppl.get_group(key).tail(int(cut_off[key]))['szp'].mean()
    bot = ppl.get_group(key).head(int(cut_off[key]))['szp'].mean()
    t_df = pd.DataFrame({'inn': [key], 'val': [top / bot]})
    val_1_7 = pd.concat([val_1_7, t_df])

In [335]:
print_df(val_1_7, 'val_1_7')

In [339]:
cnt1 = res[res.job_feb == 'Учитель'][['inn', 'szp']].groupby('inn').mean().rename(columns={'szp':'cnt1'})
cnt2 = res[res.job_feb == 'Учитель'][['inn', 'szp']].groupby('inn').quantile(0.5).rename(columns={'szp':'q'})
cnt2 = pd.merge(cnt2, res[res.job_mar == 'Учитель'][['inn', 'snils', 'szp']], how='right', on='inn')
teach_perc_out = cnt2
cnt2 = cnt2[cnt2.szp <= cnt2.q][['inn', 'szp']].groupby('inn').mean().rename(columns={'szp':'cnt2'})
val_15 = pd.merge(cnt2, cnt1, how='left', on='inn')
val_15['val'] = val_15.cnt1 / val_15.cnt2
val_15[val_15.val >= 1.4]

,cnt2,cnt1,val
inn,,,


In [340]:
print_df(val_15, 'val_15')
print_df(teach_perc_out, 'val_15_ppl')

In [239]:
def summ_isp(row, months):
    sum = 0
    for month in months:
        if row['job_' + month] in isp:
            sum += row['sum_' + month]
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum

In [250]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')) & (df.inn != 9718071371)]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [341]:
prep = res.copy()
prep['summ'] = prep.apply(lambda row: summ_isp(row, months), axis=1)
val_1_3_ppl = prep[(prep.summ.isna() == False) & (prep.summ > 0)]
val_1_3 = prep[['inn'] + ['sum_' + month for month in months]].groupby('inn').sum().reset_index()
val_1_3['sum'] = val_1_3.sum_jan + val_1_3.sum_feb
prep = prep[['inn', 'summ']].groupby('inn').sum().reset_index()
val_1_3 = pd.merge(prep[['inn', 'summ']], val_1_3[['inn', 'sum']], how='left', on='inn')
val_1_3['ans'] = val_1_3['summ'] / val_1_3['sum']
val_1_3

,inn,summ,sum,ans
0,5003021368,1.575554e+08,1.273312e+08,1.237367
1,5003021495,1.757665e+08,1.434321e+08,1.225433
2,5003021640,9.751068e+07,1.146649e+08,0.850397
3,5003096290,8.557969e+07,7.899096e+07,1.083411
4,5030032182,4.290982e+07,3.618608e+07,1.185810
...,...,...,...,...
591,7743940590,6.302232e+07,5.183528e+07,1.215819
592,9705101759,0.000000e+00,7.857748e+07,0.000000
593,9715206976,6.764273e+07,6.658418e+07,1.015898
594,9715217689,3.874152e+07,3.426072e+07,1.130785


In [342]:
print_df(val_1_3, 'val_1_3')
print_df(val_1_3_ppl, 'val_1_3_ppl')

In [344]:
len(val_1_3[val_1_3.ans < 0.75])

52

In [308]:
res[res.snils == '016-412-327 06']

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped,szp_teach
13566,7712022159,016-412-327 06,129341.55,Учитель,139859.09,Учитель,134600.32,134600.32,134600.32


In [79]:
out = res[['inn', 'szp_ped']].groupby('inn').count().rename(columns={'szp_ped': 'Количество педов'})
out

,Количество педов
inn,
5003021368,433
5003021495,447
5003021640,371
5003096290,285
5030032182,126
...,...
7743939315,42
7743940590,141
9705101759,0


In [92]:
out = df[df.job.isin(['Учитель'])].drop_duplicates(['inn', 'snils'])[['inn', 'snils']].groupby('inn').count().rename(columns={'snils':'Количество педов'})

In [94]:
out
wb = Workbook()
ws = wb.active
for  i in df_to_row(out, header=True):
    ws.append(i)
wb.save('teach.xlsx')

In [ ]:
school_sum = sample[['inn', 'sum']].groupby('inn').sum()

In [ ]:
school_sum

In [ ]:
s = sums.groupby(['snils']).count().reset_index()
wb = Workbook()
ws = wb.active
for  i in df_to_row(s, header=True):
    ws.append(i)
wb.save('test.xlsx')